# Phys 260: Python assignment header

### (1) Fill out the cell below.  
The cell below is a **code cell**.  Fill out your University of Michigan uniqname, then your name, and collaborators in the cell below **inside the quotes**.  

**Do not delete the quotes.**  We will use this information to organize your assignments.  To edit and execute cells, double click inside the cell, type, and press \<shift\>+\<enter\> to execute.

In [ ]:
UNIQNAME = ""
NAME = ""
COLLABORATORS = ""

### (2) Check your python version.  
**Execute the cell below** (double click in the cell and press \<shift\>+\<enter\>, or click in the cell and press the Run button) to check that you are using a version of python that is compatible with the tool we are using to grade your assignments.  If your ```IPython``` version is too old, we will *not* be able to grade your assignments.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

### (3) Do your best to answer all questions in the assignment.  
To answer questions, **replace** anything that says either
- "YOUR ANSWER HERE" 
- 
```
YOUR CODE HERE
raise NotImplementedError
``` 

with your answer/code.  Cells with either of the two bullet points above are cells of the notebook that will be graded.

**To edit markdown** cells (e.g. this one),  *double click in the cell to type*.  Press \<shift\>+\<enter\> to execute the cell.  Try editing the text below to replace the with your information:  

[first name] [last name], uniqname


### (4) Make sure your notebook runs sequentially.
After you complete this assignment, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Phys 260 Python Tutorial/HW 7: Integrating the Biot-Savart Law  (30 points total)

## Tutorial Summary
- Brief re-review of the Biot-Savart law and the corresponding functions
- Mutliple line wires

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

## Brief Review of the Biot-Savart Law

The <b>Biot-Savart Law</b> describes how currents produce magnetic fields: 
\begin{equation}
       \mathbf{B}(\vec{r}) = \frac{\mu_0}{4\pi}\int\frac{I\,d\vec{\ell}\times ({\vec{r}-\vec{r}^{\prime})}}{|r-r^\prime|^3}
\end{equation}
In this expression, $\vec{r}$ is a field point (we use a `np.meshgrid` to sample field points), and the integral runs over all the currents, whose positions are labeled by the vector $\vec{r}^\prime$.  Recall, when discretized for numerical calculations, integrals act as sums.

The field point $\vec{r}$ is fixed for a particular integration (sum). In general, this integral is difficult to evaluate analytically. This is why most introductory courses, like Phys260, typically restrict to examples with symmetries, like infinite straight wires, or the axis of symmetry of a loop. Computers have no such limitations, as we can sample over any configuration, regardless of symmetry (recall how we calculated the electric field due to a rectangular prism of charge).

Steps of simulating the magnetic field:
- Identify field points you wish to sample for visualization 
- Discretize the current-containing wire configuration
- Apply the numerical version of the Biot-Savart Law
- Visualize

### Functions that build the Biot-Savart Law (Group talk through - 10min)

Below is a function `calculate_magnetic_field_at_point`, the numerical calculation of the integrand in the Biot-Savart Law.  This is the magnetic field contribution due to a given current element to a single point in the field r,

\begin{equation}
d\vec{B}(\vec{r})=\frac{\mu_0}{4\pi} \frac{I\,d\vec{l}\times ({\vec{r}-\vec{r}^{\prime})}}{|r-r^\prime|^3}
\end{equation}

We can apply this function to *all* points we are sampling in our field with `np.apply_along_axis`, which is wrapped by the function `calculate_bfield_on_grid`.  

We can then account for all current elements and sum, and do this in `calculate_bfield_from_current_elements`.  This is effectively doing the integral (but quantizing as a sum),
\begin{eqnarray}
\vec{B}(\vec{r}) &=& \int d\vec{B}(\vec{r})\\
&=&\frac{\mu_0}{4\pi} \int\frac{I\,d\vec{l}\times ({\vec{r}-\vec{r}^{\prime})}}{|r-r^\prime|^3}\\
&\approx&\frac{\mu_0}{4\pi} \sum_i I\frac{\Delta \vec{l_i}\times ({\vec{r}-\vec{r_i}^{\prime})}}{|r-r_i^\prime|^3}
\end{eqnarray}
where we have individual pieces of the wire, $\Delta \vec{l_i}$, at position $\vec{r_i}^{\prime}$, and we have to sum up all of the quantized contributions.  You will notice the explicit accounting of the $\Delta l_i$ as an argument below.  Note, we are still setting constants out in front to 1 to keep plotted numbers simple.  If we assume constant current I throughout a single connected wire element, we can factor that out as well (and for simplicity set this to 1).  

In [ ]:
def calculate_magnetic_field_at_point(field_position, current_position, current_method, dl) :
    '''Calculates magnetic field at point due to current vector element at a given position
    Parameters
    ----------
    field_position : n-darray
        position of field element
    current_position : n-darray
        position of current element
    current_method : func
        callable that returns the current at a position
    dl : float
        length of wire segment, delta l
    
    '''
    idl = current_method(current_position, dl)
    r = field_position - current_position
    r_magnitude = np.linalg.norm(r)
    return np.cross(idl, r) / r_magnitude**3


def calculate_bfield_on_grid(current_position, field_positions, current_method, dl) :
    """ Find the bfield on a grid of field points due to a single current element.

    Inputs:
    current_position (n-darray) : x, y, and z position for charge, shape (3,) 
    field_positions (n-darray) : x, y, and z positions for field points, shape (3,l,m,n)
    current_method (func) : callable to return the current element at current_position
    dl (float) : length of wire segment, delta l
    
    Outputs:
    vector_bfield (n-darray) : x, y, z components of the b-field at the point field_position, shape (3,l,m,n) 

    """
    assert(current_position.shape[0]==3)
    assert(field_positions.shape[0]==3)
    return np.apply_along_axis(calculate_magnetic_field_at_point, 0, 
                                field_positions, current_position, current_method, dl)

def calculate_bfield_from_current_elements(current_positions, field_positions, current_method, dl) : 
    """ 
     Find the bfield on a grid of field points due to a single charge.

    Inputs:
    current_positions (n-darray) : x, y, and z position for charge, shape (3,n,m,l) 
    field_positions (n-darray) : x, y, and z positions for field points, shape (3,i,j,k)
    current_method (func) : callable to return the current element at single current_position
    dl (float) : length of wire segment, delta l

    Outputs:
    vector_bfield (n-darray) : x, y, z components of the b-field at the point field_position, shape (3,i,j,k)
    
    """
    assert(current_positions.shape[0] == 3)
    
    ## Checking for meshgrid input for current_positions and changing it to an array of shape (3,total_points) 
    if current_positions.ndim == 4:
        current_positions  = np.vstack([current_positions[0,:,:,:].ravel(),
                                current_positions[1,:,:,:].ravel(),
                                current_positions[2,:,:,:].ravel()])
    
    bfield_vectors_along_axis = np.apply_along_axis(calculate_bfield_on_grid, 0, 
                                                    current_positions, 
                                                    field_positions, current_method, dl)
    summed_bfield = bfield_vectors_along_axis.sum(axis=4)
    return summed_bfield

### The crux of the simulating the magnetic field for different (uniform) current configurations

Now that we have functions to calculate the magnetic field at any point due to a current carrying wire configuration, the remaining steps are to:
- Define `field_positions`, the output of a numpy meshgrid, in planes that are informative to us (you can do the calculation on a meshgrid sampling 3-dimensional space, but can add up to a lot of points and you'll only ever need to examine the field in a given slice),
- Define the points sampling the current, `current_positions`, potentially an output of a numpy meshgrid, or simply collected coordinates,
- Define a function that outputs the vector corresponding to the current sampled, `current_method`.

**Note**: Depending on how many "pieces" we break a current-carrying wire into, we'll have to account for this with the dl.  e.g. A loop wire that we quantize into 20 pieces, or that is sampled by 20 equally spaced points, will have a $d\vec{l}$ of length $2\pi R$/20.  Then, we would add up all 20 contributions to each field point.

### Define field positions (4 points -- in groups 5 min)

In the cell below, define `xy_plane_points` and `yz_plane_points` using `np.meshgrid`, indexing with 'ij'.  There is an analogous example above when we defined test_field_points.  The field positions you define should respectively sample a square plane of points where $z=0$ and $x=0$.  Sample 10 points across in a square of side length 4 (between -2 and 2), so you have a total of 100 points sampled on each plane.  Take a look at the assertion statements in the next cell over - do those shapes make sense to you?

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(xy_plane_points.shape == (3,10,10,1))
assert(yz_plane_points.shape == (3,1,10,10))

### Define wire positions (3 points -- in groups 5 min)

For the preflight, we defined wire positions for a line wire, `points_of_linewire`, where the linewire ran along the x-axis.  One possible solution to sample a current with this configuration with 100 sampled points is,

```
xarray, yarray, zarray = np.meshgrid(np.linspace(-5,5,100),
                                    np.array([0]),
                                    np.array([0]), indexing='ij')
points_of_linewire = np.array([xarray, yarray, zarray])
```

In the cell below, define sampled positions for two line wires, `parallel_points_linewires`, where we have two linewires running parallel to the x-axis, one at $y=0.5m$, the other at $y=-0.5m$.  The shape of `parallel_points_linewires` should show that the first axis (zeroeth) has 3 elements across, corresponding to the x, y, and z coordinates of each point sampling the wires.

There are a couple of ways to accomplish this, one is to use meshgrid with a different y array than the example above. You can also define two separate meshgrid arrays, and to use the `np.concatenate`. Concatenation will be more useful for more complicated current configurations.

In [ ]:
# Define parallel_points_linewires here

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(parallel_points_linewires.shape[0] == 3)
assert(parallel_points_linewires[0].min() == -5)
assert(parallel_points_linewires[1].min() == -0.5)
assert(parallel_points_linewires[1].max() == 0.5)
assert((parallel_points_linewires[2] == 0).all())

### Define a method to determine current (3 points -- in groups 5 min)

Let us now define a method for anti-parallel wires.  We want the current to point in the "+x" direction in the wire above the zx-plane (y=0.5), and in the "-x" direction in the wire below the zx-plane (y=-0.5).  **Quick question for the class before break out:** What should the type of the returned quantity be (fill out the docstring)?

In [ ]:
def anti_parallel_xwires(current_position, dl) :
    """Returns the Id\vec{l} of a wire running parallel to the x-axis, 
    with positive current above the zx-plane, negative below.    

    Inputs:
    current_position (3, n-darray) : x, y, and z position for current element 

    Outputs:
    idl (3, n-darray) : x, y, z values of the current * dl
    """
    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert((anti_parallel_xwires(np.array([1,1,0]), .1) == np.array([.1,0,0])).all())
assert((anti_parallel_xwires(np.array([-1,-1,0]), .1) == np.array([-.1,0,0])).all())

### Let's visualize the system (in class discussion/work together -- 5 points total)

**Quick question for class:**  What slice (plane) of field space should we visualize that might be interesting for two anti-parallel wires?

We will talk through using `calculate_bfield_from_current_elements` to define `bfield_slice_antiparallel` below.

In [ ]:
# Calculate the b-field on that slice due to antiparallel wires

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(bfield_slice_antiparallel.shape[0] == 3)

**Preparing arrays  for plot (2 points):** Let's prepare the arrays to use in streamplot to plot the magnetic field in the yz-plane. While our calculation takes place in 3D, we need to separate the meshgrid arrays into their individual components to plot them in 2D. Grab the correct 'slices' of the yz_plane_points and bfield_slice_antiparallel, using indexes. Please define the following variables for plotting:

- `z_pts_plot` and `y_pts_plot` : 2-D arrays containing the z and y field points for streamplot.
- `bfield_z_antiparallel` and `bfield_y_antiparallel` : 2-D arrays containing the z and y components of the calculated magnetic field for streamplot.
- `bfield_mag_antiparallel` : 2-D array containing the magnitude of the magnetic field, for coloring purposes. If you use linalg.norm, make sure to check the shape of the output!

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert(z_pts_plot.ndim==2)
assert(bfield_z_antiparallel.shape==z_pts_plot.shape)
assert(bfield_z_antiparallel.shape==z_pts_plot.shape)

**Visualize (class discussion) (2 points):** Use streamplot to plot the magnetic field in the zy plane. Because of the indexing of streamplot, please plot z on the 'x-axis' and y on the 'y-axis'. How might you indicate the direction of the current in the figure below?  Add a useful visual (see example with 'x' and 'o' from the preflight). Assign the streamplot output to `strm` for the colorbar call.

In [ ]:
fig, ax1 = plt.subplots(1, figsize=(8,8))

color = np.log10(bfield_mag_antiparallel)   # colors the arrows based on field strength

# YOUR CODE HERE
raise NotImplementedError()

fig.colorbar(strm.lines)

ax1.set_aspect('equal')
ax1.set_xlabel('z', fontsize=16)
ax1.set_ylabel('y', fontsize=16)

**Discuss the plot (Take home)**:  Does this figure make sense?  Make sure to add an "x" and an "o" symbol corresponding to current going into and out of the page at the appropriate location.  Why does this make sense?

YOUR ANSWER HERE

## Homework Summary (13 points)
- Mutliple line wires (add three above and three below the zx-plane) -- need to define the `current_positions` array corresponding to `four_parallel_points_linewires`
- Multiple line wires with parallel current -- need to define a new (but very similar to `anti_parallel_xwires`) function called `parallel_xwires`. 
- A loop wire -- need to use new `np` tools, `np.cos` and `np.sin` to define the $d\vec{l}$ in the function for current, also need to sample points around a circle to define the loop wire positions.
- Multiple loop wires -- analogs from all of the above

### Four wires above and below the zx-plane (2 points)

In the cell below, define `four_parallel_points_linewires` all running parallel to the x-axis.  The first four should sample four parallel wires at $y=0.5$, at equidistant z-values between $-1<z<1$ and four parallel wires at $y=-0.5$, at the same equidistant z-values.  This is an extension of `parallel_points_linewires`. I recommend defining a list of your wire's z-values, `zvalues`, and using this both in defining the points in the linewires and also in marking the "x"'s and "o"'s of your streamplot figure.

In [ ]:
# Define four_parallel_points_linewires here

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(four_parallel_points_linewires.shape[0]==3)
assert(four_parallel_points_linewires[1].max() == 0.5)
assert(four_parallel_points_linewires[1].min() == -0.5)

Plot the magnetic field in the yz-slice for the antiparallel current case.  Add the appropriate "x"'s and "o"'s using `zvalues`.  Do you see what the magnetic field behavior might look like if we had, instead, sheets of antiparallel current?  You can experiment with: 
- yz plane points further towards the ends of the wires (e.g. $x\approx5$ where you see the edge effects of our discretization (these are not truly infinitely long wires),
- moving the wires closer together to see what happens when they sit at $y=\pm 0.25$ (move them back before submitting, otherwise the assertion cell will fail)
- Note: Leave a figure with the original prompt's configuration that illustrates the B-field in the previously defined `yz_plane_points`. 

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

### A loop wire (2 points) 

Defining both the wire positions and $\Delta \vec{l}$ is a bit trickier here because you are dealing with a circle.  The following text should walk you through this construction.  

Consider a circle on the xy-plane (constant z, say $z=0$), with current flowing clockwise (so with increasing $\phi$).  With the line wire, we varied over x, so $d\vec{l}=dx\hat{x}$, coded with `dl*np.array([1,0,0])`.  This problem has cylindrical symmetry, so we will work in cylindrical coordinates, ($r,\phi,z$), and our $d\vec{l}$ will vary with our $d\phi\hat{\phi}$. 

For a circle with radius R (circumference $S=2\pi R$), quantized into n arclets of length $\Delta s=2\pi R/n$ at each $\phi_i$ (where $1\leq i\leq n$ so $\Delta\phi/2\pi=1/n$, we have a corresponding $d\vec{l}$ of,
\begin{eqnarray}
d\vec{l}&=&dl\hat{\phi}\\
&=&Sd\phi\hat{\phi}\\
&\approx&S\frac{\Delta\phi}{2\pi}\hat{\phi}\\
&=&\Delta s\hat{\phi}\\
&=&\frac{2\pi R}{n}\hat{\phi}\\
\end{eqnarray}
namely the arclength times the unit vector tangent to the circle.  So, for the ith segment,
\begin{eqnarray}
\Delta\vec{l}_i&=&\frac{2\pi R}{n}\hat{\phi_i}
\end{eqnarray}
The conversion of $\hat{\phi}$ to cartesian coordinates will depend on the value of $\phi_i$ corresponding to the sampled position in the loop.  If you have not yet seen the conversions between unit vectors, the [wiki page](https://en.wikipedia.org/wiki/Del_in_cylindrical_and_spherical_coordinates) is fairly comprehensive, but remember to swap $\theta\leftrightarrow\phi$ to go between math $\leftrightarrow$ physics norms.
\begin{eqnarray}
\hat{\phi}_i&=&-\sin\phi_i\hat{x}+\cos\phi_i\hat{y}\\
&=&\frac{-y\hat{x}+x\hat{y}}{R}\\
\end{eqnarray}
The positions of points sampled in the loop, ($x_i,y_i,z$), correspond to the individual $\phi_i$'s (where $1\leq i\leq n)$, and these are,
\begin{eqnarray}
x_i&=&R\cos\phi_i\\
y_i&=&R\sin\phi_i
\end{eqnarray}

In the cell below, define both the numpy array that samples `points_in_loop` and the function, `clockwise_loop_wire_xy`, that returns your $d\vec{l}$ for a given point in a loop parallel to the xy-plane with current running clockwise.  Assume a radius of $R=1m$ to again, keep numbers simple and reduce a proliferation of constants out in front.  It may be useful to know that numpy has both [`np.cos`](https://numpy.org/doc/stable/reference/generated/numpy.cos.html) and [`np.sin`](https://numpy.org/doc/stable/reference/generated/numpy.sin.html) functions, which do expect angles in radians.  

I suggest you first quantize sampled values of $\phi$ in the same manner we quantized sampled values of $x$ in the line wire example (make sure to *not* double count 0 and $2\pi$).  If you use `np.linspace`, there is a key word argument `endpoint` that is useful.

In [ ]:
# Define the numpy array points_in_loop and the function clockwise_loop_wire here.

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(points_in_loop.shape[0] == 3)
assert((points_in_loop >= -1).all() & (points_in_loop <= 1).all())
assert((points_in_loop[2] == 0).all())

**Plot (2 points)** the magnetic field of your loop of current in the yz-plane.  When you call `calculate_bfield_from_current_elements`, you will need to use the appropriate value for `dl`.  Try shrinking the radius of the loop to see what happens (you should be able to do this by simply multiplying by a constant when you define `points_in_loop`.  You can also try multiplying `dl` to keep current consistent, but simply varying `points_in_loop` suffices for visualization.

In [ ]:
# Plot the magnetic field due to a loop here

# YOUR CODE HERE
raise NotImplementedError()

### Helmholz coils (3 points)

You may have seen Helmholz coils in your lab class, a configuration of two parallel loops to produce a relatively uniform magnetic field in between the coils.  For one of your written homeworks, you wrote down an expression for $B(z)$ for two parallel circular loops with radius $R$, separated by a distance $a$. You can solve for $a$ such that $B(z)$ is uniform near the midpoint between the two coils. This means that you are looking for the value of $a$ that makes,
\begin{eqnarray}
\frac{d^2B}{dz^2}|_{z=0}&=&0
\end{eqnarray}
This is the case when $a=R$, so the coils are as far apart as the coil radius. Helmholz coils are meant to create a relatively uniform magnetic field in the region between the loops.

Set up Helmholz coils below, defining `points_in_helmholz` that sit at $z=\pm 0.5$, so $d=1=R$.

In [ ]:
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
"""Execute to check you're on the right track"""
assert(points_in_helmholz.shape[0] == 3)
assert((points_in_helmholz >= -1).all() & (points_in_helmholz <= 1).all())
assert((np.abs(points_in_helmholz[2]) == 0.5).all())

**Plot** the magnetic field in the yz=plane.  Both loop currents go in the clockwise direction.

In [ ]:
# Plot the magnetic field due to a helmholz coils here


# YOUR CODE HERE
raise NotImplementedError()

**Plot a 1D linecut(2 points):** 

While 2D plots such as streamplots or false color images can convey a lot of information, often 'line cuts' or 'profiles' along 1D cuts can give easier to interpret or more quantitative information about whatever we are plotting. 


- The magnitude of the magnetic field along the z-axis ($x,y=0$) through the axis of the coils.
- The magnitude of the magnetic field along the y-axis ($x,z=0$) in between the two coils. 

While we *could* use the field we've already calclulated, the 1D profile will only have 10 points and not fully capture the shape of the magnetic field. 

Set up two separate meshgrid arrays for the profiles along z and y, plotting 100 points between [-2,2] and then use 
calculate_bfield_from_current_elements to find the field. 

Overplot both lines on the same axis, but label the lines and add a legend. As with the 2D plots, the shape of the bfields along the lines will be wrong for a normal plot. You can use array indexes to fix the shape, but another useful function for this type of operation is <a href='https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html'>np.squeeze</a>. Squeeze removes all dimensions of an array with length one. 

In [ ]:
# Calculate higher density lines along the z and y axes, Plot the magnitude of the magnetic field along them. 
# YOUR CODE HERE
raise NotImplementedError()

**Discuss the plot and data (2 points)**: Relate the shape of the magnetic field curves plotted in the linecuts to the 2D field you plotted earlier. What are the differences between the curves along the y and z axes?

Let's say we wanted to show how uniform the field inside the Helmholz coil, tell me how you might modify the plot to better show this. 

YOUR ANSWER HERE